In [6]:
import time
import random
import os
import subprocess
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import openpyxl
from bs4 import BeautifulSoup

# =============================================================================
# 1. (Optional) Kill leftover Edge processes (Windows-only)
# =============================================================================
subprocess.run(["taskkill", "/im", "msedge.exe", "/f"], check=False)
subprocess.run(["taskkill", "/im", "msedgedriver.exe", "/f"], check=False)
time.sleep(8)

# =============================================================================
# 2. Configure Edge Options
# =============================================================================
edge_options = Options()

# 2.1. Specify your Edge "User Data" directory
# Example: C:\Users\<YourName>\AppData\Local\Microsoft\Edge\User Data
edge_options.add_argument(r"--user-data-dir=C:\Users\aminj\AppData\Local\Microsoft\Edge\User Data")

# 2.2. Specify which profile subfolder: "Default", "Profile 1", "Profile 2", etc.
edge_options.add_argument("--profile-directory=Profile 1")

# 2.3. Helpful flags if you have "DevToolsActivePort" or crashes
edge_options.add_argument("--no-sandbox")
edge_options.add_argument("--disable-dev-shm-usage")

# 2.4. Uncomment for headless mode if you don't need to see the browser:
# edge_options.add_argument("--headless")
# edge_options.add_argument("--disable-gpu")

# =============================================================================
# 3. Launch Edge WebDriver
# =============================================================================
driver = webdriver.Edge(options=edge_options)

try:
    # =========================================================================
    # 4. Navigate to the target Twitter (X) profile
    # =========================================================================
    driver.get("https://twitter.com/Aminaminnii")  # Change to whichever user
    time.sleep(10)  # Let initial tweets load

    # =========================================================================
    # 5. Scroll multiple times with random pauses and collect tweet data
    # =========================================================================
    SCROLL_TIMES = 30
    results = []

    for _ in range(SCROLL_TIMES):
        # Scroll down
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        # Random pause between 2 and 5 seconds
        time.sleep(random.uniform(2, 5))

        # Collect all tweet <article> elements
        articles = driver.find_elements(By.CSS_SELECTOR, "article[data-testid='tweet']")

        for article in articles:
            # -----------------------------------------------------------------
            # (A) Tweet URL & ID
            # -----------------------------------------------------------------
            link_elems = article.find_elements(By.XPATH, ".//a[contains(@href, '/status/')]")
            if not link_elems:
                continue
            tweet_url = link_elems[0].get_attribute("href")
            tweet_id = tweet_url.split("/")[-1] if "/status/" in tweet_url else ""

            # -----------------------------------------------------------------
            # (B) Author Name (the displayed name, not necessarily @handle)
            # -----------------------------------------------------------------
            # Typically under: div[data-testid='User-Name'] > div > span
            # We'll do a broader approach, then parse the text.
            author_elems = article.find_elements(By.CSS_SELECTOR, "div[data-testid='User-Name'] span")
            account_name = author_elems[0].text.strip() if author_elems else ""

            # -----------------------------------------------------------------
            # (C) Tweet Time (from <time datetime="...">)
            # -----------------------------------------------------------------
            time_elems = article.find_elements(By.XPATH, ".//time")
            tweet_time = time_elems[0].get_attribute("datetime") if time_elems else ""

            # -----------------------------------------------------------------
            # (D) Tweet Text (including embedded links)
            # -----------------------------------------------------------------
            text_elems = article.find_elements(By.CSS_SELECTOR, "div[data-testid='tweetText']")
            if not text_elems:
                continue
            html_content = text_elems[0].get_attribute("innerHTML")

            # Parse HTML with BeautifulSoup
            soup = BeautifulSoup(html_content, "html.parser")
            full_text_parts = []
            for node in soup.descendants:
                if node.name == "a":
                    href = node.get("href")
                    link_text = node.get_text(strip=True)
                    if link_text:
                        full_text_parts.append(f"{link_text} ({href})")
                elif node.string:
                    text_str = node.string.strip()
                    if text_str:
                        full_text_parts.append(text_str)
            content = " ".join(full_text_parts).strip()

            # If we have at least an ID and some content, store it
            if tweet_id and content:
                results.append((tweet_id, tweet_url, account_name, tweet_time, content))

finally:
    # =========================================================================
    # 6. Quit Edge
    # =========================================================================
    driver.quit()

# =============================================================================
# 7. Write Data to Excel
# =============================================================================
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Tweets"

# Header row
sheet.append(["TweetID", "TweetLink", "AccountName", "Time", "Content"])

for (tid, url, name, ttime, txt) in results:
    sheet.append([tid, url, name, ttime, txt])

excel_filename = "tweets_edge.xlsx"
workbook.save(excel_filename)

print(f"Done! Extracted {len(results)} tweets. Saved to {excel_filename}.")


Done! Extracted 308 tweets. Saved to tweets_edge.xlsx.


In [7]:
import snscrape.modules.twitter as sntwitter

username = "Aminaminnii"
max_count = 2000

query = f"from:{username} filter:retweets"
tweets_data = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i >= max_count:
        break
    tweets_data.append((tweet.id, tweet.date, tweet.user.username, tweet.content))

print(f"Got {len(tweets_data)} retweets from snscrape.")


Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22from%3AAminaminnii%20filter%3Aretweets%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22vi

ScraperException: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22from%3AAminaminnii%20filter%3Aretweets%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22